In [11]:
from rdflib import Graph, URIRef, RDF

KG_PATH = r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\Test2_filled.ttl"
g = Graph()
g.parse(KG_PATH, format="turtle")

Q = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ap:  <http://www.semanticweb.org/AgentProgramParams/>

SELECT ?pou ?name ?lang ?code
WHERE {
  ?pou rdf:type ?t .
  FILTER(?t IN (ap:class_Program, ap:class_FBType)) .
  ?pou ap:dp_hasPOUName ?name .
  ?pou ap:dp_hasPOUCode ?code .
  FILTER(CONTAINS(STR(?code), "METHOD")) .
  OPTIONAL { ?pou ap:dp_hasPOULanguage ?lang . }
}
"""
rows = []
for r in g.query(Q):
    rows.append({
        "pou_iri": str(r.pou),
        "name": str(r.name),
        "lang": str(r.lang) if r.lang else None,
        "code": str(r.code) if r.code else None,
    })
    print(str(r.pou), str(r.name), str(r.lang) if r.lang else None, str(r.code))

print("POUs:", len(rows))


http://www.semanticweb.org/AgentProgramParams/FBType_FB_MyOpcUaMethod FB_MyOpcUaMethod ST // METHOD Multiply of FB_MyOpcUaMethod
Multiply := a * b;
http://www.semanticweb.org/AgentProgramParams/FBType_JobMethode_Schablone JobMethode_Schablone ST // POU JobMethode_Schablone body
JobMethode_Name(
    MethodCall := MethodCall_Name,
    NotAus_Signal := NotAusSignal,
    Direction_01 := Direction_01,
    Direction_02 := Direction_02,
    LightSensor_01 := LightSensor_01,
    LightSensor_02 := LightSensor_02,
    TS_Endschalter_01 := TS_Endschalter_01,
    AutomaticMode := _REFACTOR_,
    TS_Endschalter_02 := TS_Endschalter_02,
    TS_EndschalterEntlastung := TS_EndschalterEntlastung,
    TOF_Time_Motoransteuerung := TOF_Time_Motoransteuerung,
    TOF_Time_EndschalterEntlastung := TOF_Time_EndschalterEntlastung
);
Motor_Ansteuerung01 := JobMethode_Name.Motor_Direction01;

// METHOD Abort of JobMethode_Schablone
callCounterAbort := callCounterAbort + 1;

// METHOD CheckState of JobMethode_Sc

In [12]:
code = rows[0]["code"] or ""

print("Hat echte Newlines?", "\n" in code)
print("Hat escaped Newlines?", "\\n" in code)
print("Count '// METHOD':", code.count("// METHOD"))

# Zeig die Stelle rund um den ersten METHOD-Marker als repr (zeigt unsichtbare Zeichen)
idx = code.find("// METHOD")
print("idx:", idx)
print(repr(code[max(idx-10,0):idx+30]))


Hat echte Newlines? True
Hat escaped Newlines? False
Count '// METHOD': 1
idx: 0
'// METHOD Multiply of FB_MyOpc'


In [8]:
code = rows[0]["code"]
i = code.find("METHOD")
print(code[i-80:i+200])
print(repr(code[i-20:i+40]))

// METHOD Multiply of FB_MyOpcUaMethod
Multiply := a * b;
'ult'


In [10]:
import re
import hashlib

METHOD_RE = re.compile(
    r"(?ms)^\s*//\s*METHOD\s+(?P<method>\w+)[^\r\n]*\r?\n"   # Header: METHOD <Name> ... bis Zeilenende
    r"(?P<body>.*?)(?=^\s*//\s*METHOD\b|\Z)"                 # Body: bis nächste METHOD Zeile oder String-Ende
)

def sha(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8")).hexdigest()[:12]

def extract_methods(code: str):
    if not code:
        return []
    # Normalize line endings (optional, aber hilfreich)
    code = code.replace("\r\n", "\n").replace("\r", "\n")

    out = []
    for m in METHOD_RE.finditer(code):
        body = m.group("body").strip()
        out.append({
            "method": m.group("method"),
            "body_excerpt": "\n".join(body.splitlines()[:25]),
            "body_hash": sha(body),
            "body_len": len(body),
        })
        print(f"Found method: {m.group('method')} (len={len(body)})")
    return out



In [ ]:
from groq import Groq
import json

client = Groq(api_key=groq_api_key)

SKILL_SCHEMA = {
  "name": "skill_extraction",
  "strict": False,  # True geht nur auf Modellen, die strict unterstützen; sonst 400 -> dann False nutzen
  "schema": {
    "type": "object",
    "additionalProperties": False,
    "properties": {
      "skills": {
        "type": "array",
        "items": {
          "type": "object",
          "additionalProperties": False,
          "properties": {
            "skill_name": {"type": "string"},
            "pou": {"type": "string"},
            "method": {"type": "string"},
            "intent_summary": {"type": "string"},
            "inputs": {"type": "array", "items": {"type": "string"}},
            "outputs": {"type": "array", "items": {"type": "string"}},
            "preconditions": {"type": "array", "items": {"type": "string"}},
            "postconditions": {"type": "array", "items": {"type": "string"}},
            "confidence": {"type": "number"}
          },
          "required": ["skill_name", "pou", "method", "intent_summary", "inputs", "outputs", "preconditions", "postconditions", "confidence"]
        }
      }
    },
    "required": ["skills"]
  }
}

SKILL_DEFINITION = """
Definition Skill (für dieses Projekt):
Ein Skill ist eine ausführbare Job-Methode in einem POU (Start/Abort/CheckState/...),
die eine klar benennbare Funktion an der Ressource ausführt.
Der Skill hat Inputs/Outputs (Parameter oder verwendete Variablen), und idealerweise
Preconditions/Postconditions (z.B. Sensorzustände).
"""

def llm_extract_skills_for_pou(pou_card: dict):
    prompt = f"""{SKILL_DEFINITION}

Aufgabe:
- Analysiere die folgenden Kandidaten (POU + Methoden-Ausschnitte).
- Erzeuge pro Methode ein Skill-Objekt, falls es ein Skill ist.
- Wenn eine Methode kein Skill ist: ignoriere sie.
- Antworte NUR als JSON nach Schema.

Kandidaten:
{json.dumps(pou_card, ensure_ascii=False, indent=2)}
"""
    resp = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        response_format={"type": "json_schema", "json_schema": SKILL_SCHEMA},
        temperature=0.0,
        max_completion_tokens=800,
    )
    return json.loads(resp.choices[0].message.content)

# Beispiel: erster POU
result = llm_extract_skills_for_pou(candidates[0])
print(json.dumps(result, ensure_ascii=False, indent=2))
